## Exploration Exercises

Answer the questions below by using a combination of the techniques discussed in the lesson that you think is appropriate.

- How many different cases are there, by department?
- Does the percentage of cases that are late vary by department?
- On average, how late are the late cases by department?
- What is the service type that is the most late? Just for Parks & Rec?
- For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.
- Does whether or not its a weekend matter for when a case is opened/closed?
- On average, how many cases are opened a day for the Customer Service department?
- Does the number of service requests for the solid waste department vary by day of the week?

In [ ]:
# How many different cases are there, by department?

In [ ]:
# Does the percentage of cases that are late vary by department?

In [ ]:
# On average, how late are the late cases by department?

In [ ]:
# What is the service type that is the most late? Just for Parks & Rec?

In [ ]:
# For the DSD/Code Enforcement department, what are the most common service request types?

In [ ]:
# Does whether or not its a weekend matter for when a case is opened/closed?

In [ ]:
# On average, how many cases are opened a day for the Customer Service department?

In [ ]:
# Does the number of service requests for the solid waste department vary by day of the week?